In [1]:
import os, sys, pickle, math, warnings, time
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow_probability as tfp
from sklearn.preprocessing import StandardScaler
tfe = tf.contrib.eager
try:
    tfe.enable_eager_execution()
except ValueError:
    pass

In [2]:
tfd = tfp.distributions
tfdtype = tf.float32
npdtype = np.float32
tfnn = tf.nn
names = locals()

# SMC参数设置
J = 10    #迭代轮数
B = 10 #10**4 #并行样本数
K = 16    #每次MH的轮数

layer_size = [8, 7, 10, 1]
l2_norm = 0.0003

# 数据导入以及标准化处理
features = ['fixed acidity','volatile acidity','residual sugar','chlorides','free sulfur dioxide','density','pH','sulphates']
df = pd.read_csv('../winequality-white.csv', sep=';')
X = df[features].values
y = (df.quality >= 7).values

X = StandardScaler().fit_transform(X).astype(npdtype)

In [3]:
'''def samples_init(w_1s, b_1s, w_2s, b_2s, w_3s, b_3s):
    # fill in weights and biases with random normal
    for b in range(B):
        i = 1
        for s1, s2 in zip(layer_size[:-1], layer_size[1:]):
            names['w_'+str(i)+'s'].append(weight_init([s1, s2]))
            names['b_'+str(i)+'s'].append(bias_init([s2]))
            i += 1
    return #w_1s, b_1s, w_2s, b_2s, w_3s, b_3s
'''
def samples_init():
    # fill in weights and biases with random normal
    len_layer = len(layer_size)
    # create empty list for weights and biases
    for i in range(1, len_layer):
        names['w_'+str(i)+'s'] = []
        names['b_'+str(i)+'s'] = []
    samples = []
    samples_append = samples.append
    for b in range(B):
        theta = []
        for s1, s2 in zip(layer_size[:-1], layer_size[1:]):
            theta.append(weight_init([s1, s2]))
            theta.append(bias_init([s2]))
        samples_append(np.array(theta))
    return np.array(samples)

# 网络权重初始化
def weight_init(shape, mean=0.0, stddev=10.0):
    weights = tf.truncated_normal(shape, mean=mean, stddev=stddev, dtype=tfdtype)
    return tf.Variable(weights)

# 偏置的初始化
def bias_init(shape, mean=0.0, stddev=10.0):
    biases = tf.random_normal(shape, mean=mean, stddev=stddev, dtype=tfdtype)
    return tf.Variable(biases)

# log likelihood function
def log_likeihood_fn(w_1, b_1, w_2, b_2, w_3, b_3):
    print(w_1)
    print(b_1)
    layer_1 = tfnn.relu(tf.matmul(X, w_1) + b_1)
    layer_2 = tfnn.relu(tf.matmul(layer_1, w_2) + b_2)
    layer_3 = -tfnn.sigmoid_cross_entropy_with_logits(
        labels=y, 
        logits=(tf.matmul(layer_2, w_3) + b_3))
    print(type(tf.reduce_mean(layer_3)))
    return tf.reduce_sum(layer_3)
    
# target log probability function 
# issue: no phi_j
def smc_log_prob_fn(w_1, b_1, w_2, b_2, w_3, b_3):
    # compute log prior ratio
    pri = 0
    pri += tf.reduce_sum(w_1 * w_1)
    pri += tf.reduce_sum(w_2 * w_2)
    pri += tf.reduce_sum(w_3 * w_3)
    pri += tf.reduce_sum(b_1 * b_1)
    pri += tf.reduce_sum(b_2 * b_2)
    pri += tf.reduce_sum(b_3 * b_3)
    
    # compute log likelihood
    layer_1 = tfnn.relu(tf.matmul(X, w_1) + b_1)
    layer_2 = tfnn.relu(tf.matmul(layer_1, w_2) + b_2)
    layer_3 = -tfnn.sigmoid_cross_entropy_with_logits(
        labels=y, 
        logits=(tf.matmul(layer_2, w_3) + b_3))
    return tf.reduce_sum(layer3) + 0.0003 * pri

def draw_from_multinomial(thetas, p):
    dist = tfd.Multinomial(total_count=n, probs=p)
    idxes = dist.samples(1)[0]
    with tf.Session() as sess:
        idxes = sess.run(idxes)
    idxes = idxes.astype(np.int32)
    '''var_w_1s = []
    var_b_1s = [] 
    var_w_2s = []
    var_b_2s = [] 
    var_w_3s = []
    var_b_3s = []
    for i in range(B):
        while(samples[i] != 0):
            
            var_w_1s.append(w_1s[i])
            var_b_1s.append(b_1s[i])
            var_w_2s.append(w_2s[i])
            var_b_2s.append(b_2s[i])
            var_w_3s.append(w_3s[i])
            var_b_3s.append(b_3s[i])
            samples[i] -= 1
    return np.array(var_w_1s), np.array(var_b_1s), np.array(var_w_2s), np.array(var_b_2s), np.array(var_w_3s), np.array(var_b_3s)'''
    res = []
    res_append = res.append
    for i in range(B):
        while(idxes[i] != 0):
            res_append(thetas[i])
            idxes[i] -= 1
    return np.array(res)

In [ ]:
'''
def smc_body(w_1s, b_1s, w_2s, b_2s, w_3s, b_3s, j, n):
    start = time.time()
    # Correction
    v = []
    for (w1, b1, w2, b2, w3, b3) in zip(w_1s, b_1s, w_2s, b_2s, w_3s, b_3s):
        v.append(log_likeihood_fn(w1, b1, w2, b2, w3, b3))
    v = tf.exp((phi[j] - phi[j-1]) * np.array(v))
    
    # Selection
    w = B * v * w / tf.reduce_sum(v * w)
    ESS = B**2 / tf.reduce_sum(w * w)
    
    if ESS > 0.5*B:
        var_w_1s = deepcopy(w_1s)
        var_b_1s = deepcopy(b_1s) 
        var_w_2s = deepcopy(w_2s)
        var_b_2s = deepcopy(b_2s) 
        var_w_3s = deepcopy(w_3s)
        var_b_3s = deepcopy(b_3s)
    else:
        var_w_1s, var_b_1s, var_w_2s, var_b_2s,var_w_3s, var_b_3s =  draw_from_multinomial(w_1s, b_1s, w_2s, b_2s, w_3s, b_3s, w, n)
        
    # RWM sampling
    init_state = [var_w_1s, var_b_1s, var_w_2s, var_b_2s,var_w_3s, var_b_3s]
    samples, _ = tfp.mcmc.sample_chain(
      num_results=K,
      current_state=init_state, #the first dimension is B
      kernel=tfp.mcmc.RandomWalkMetropolis(
         target_log_prob_fn=smc_log_prob_fn,
         seed=42),
      num_burnin_steps=0,
      parallel_iterations=10)  # For determinism.
    print ('轮数：',j, '总时间：', time.time()-start)
    return samples
'''

In [11]:
def smc_body(samples, phi, w, j):
    start = time.time()
    # Correction
    v = []
    for (w1, b1, w2, b2, w3, b3) in samples:
        v.append(log_likeihood_fn(w1, b1, w2, b2, w3, b3))
    v = tf.exp((phi[j] - phi[j-1]) * np.array(v))
    
    # Selection
    w = B * v * w / tf.reduce_sum(v * w)
    ESS = B**2 / tf.reduce_sum(w * w)
    
    if ESS > 0.5*B:
        var_samples = deepcopy(samples)
    else:
        #var_w_1s, var_b_1s, var_w_2s, var_b_2s,var_w_3s, var_b_3s =  draw_from_multinomial(w_1s, b_1s, w_2s, b_2s, w_3s, b_3s, w, n)
        var_samples = draw_from_multinomial(samples, w)
        
    # RWM sampling
    #init_state = [var_w_1s, var_b_1s, var_w_2s, var_b_2s,var_w_3s, var_b_3s]
    samples, _ = tfp.mcmc.sample_chain(
      num_results=K,
      current_state=var_samples, #the first dimension is B
      kernel=tfp.mcmc.RandomWalkMetropolis(
         target_log_prob_fn=smc_log_prob_fn,
         seed=42),
      num_burnin_steps=0,
      parallel_iterations=10)  # For determinism.
    print ('轮数：',j, '总时间：', time.time()-start)
    with open('./smc4/'+str(j),'wb') as f:
            pickle.dump(samples, f)
    return samples[-1], phi, w, j+1

def smc_cond(samples, phi, w, j):
    return j<J

In [8]:
def smc(samples):
    n = len(X)
    w = np.ones(shape=B)
    phi = np.linspace(0, 1, J)**2
    j = tf.constant(0)
    samples, phi, w, j = tf.while_loop(cond=smc_cond, body=smc_body, loop_vars=[phi, w, j])
    with tf.Session() as sess:
        sess.run(samples)
    return

In [9]:
if __name__=="__main__":    
    samples = samples_init()
    smc(samples)

UnboundLocalError: local variable 'samples' referenced before assignment

In [ ]:
help(tf.while_loop)